In [1]:
import pandas as pd
import numpy as np

In [2]:
tag_counts = pd.read_csv("./tags_counts_rank.csv")

In [3]:
df = tag_counts[tag_counts['count'] > 1000].sort("avg_rank") # only top tags

In [4]:
df.reset_index(drop=True,inplace=True)

In [5]:
d = pd.read_csv("./X.csv")

In [6]:
from sklearn.manifold import TSNE as tsne
from PIL import Image

In [7]:
thumb_side = 96 # publication size

## Simple

In [8]:
'''

for tag in list(df.tag):
    tmp = d[d[tag]!=0]
    tmp.reset_index(drop=True,inplace=True)
    
    num_bin = int(round( ( len(tmp) * 48 ) / 5000 )) 
    
    X = tmp.iloc[:,1:451].as_matrix() # subset to smaller feature space
    tsne_obj = tsne(n_components=2)
    
    subspace_tsne = pd.DataFrame(tsne_obj.fit_transform(X),columns=["x","y"])
    subspace_tsne['grid_x'] = pd.cut(subspace_tsne['x'],num_bin,labels=False)
    subspace_tsne['grid_y'] = pd.cut(subspace_tsne['y'],num_bin,labels=False)
    subspace_tsne['local_path'] = tmp.local_path
    
    grid_side = num_bin
    px_w = thumb_side * grid_side
    px_h = thumb_side * grid_side
    
    canvas = Image.new('RGB',(px_w,px_h),(50,50,50))
                  
    for i in range(len(subspace_tsne)):
        im = Image.open(subspace_tsne.local_path.loc[i])
        im.thumbnail((thumb_side,thumb_side),Image.ANTIALIAS)
        x = subspace_tsne.grid_x.loc[i] * thumb_side
        y = subspace_tsne.grid_y.loc[i] * thumb_side
        canvas.paste(im,(x,y))
        
    canvas.save("/Users/damoncrockett/Desktop/stpete/viz/"+tag+"_"+str(num_bin)+".png")
    
'''

'\n\nfor tag in list(df.tag):\n    tmp = d[d[tag]!=0]\n    tmp.reset_index(drop=True,inplace=True)\n    \n    num_bin = int(round( ( len(tmp) * 48 ) / 5000 )) \n    \n    X = tmp.iloc[:,1:451].as_matrix() # subset to smaller feature space\n    tsne_obj = tsne(n_components=2)\n    \n    subspace_tsne = pd.DataFrame(tsne_obj.fit_transform(X),columns=["x","y"])\n    subspace_tsne[\'grid_x\'] = pd.cut(subspace_tsne[\'x\'],num_bin,labels=False)\n    subspace_tsne[\'grid_y\'] = pd.cut(subspace_tsne[\'y\'],num_bin,labels=False)\n    subspace_tsne[\'local_path\'] = tmp.local_path\n    \n    grid_side = num_bin\n    px_w = thumb_side * grid_side\n    px_h = thumb_side * grid_side\n    \n    canvas = Image.new(\'RGB\',(px_w,px_h),(50,50,50))\n                  \n    for i in range(len(subspace_tsne)):\n        im = Image.open(subspace_tsne.local_path.loc[i])\n        im.thumbnail((thumb_side,thumb_side),Image.ANTIALIAS)\n        x = subspace_tsne.grid_x.loc[i] * thumb_side\n        y = subspace_

## Fill

In [9]:
from shapely.geometry import Point

In [10]:
crit = np.sqrt(2) # adjacency

In [11]:
PRE = "/Users/damoncrockett/Desktop/stpete/viz/tag_fill_crit_alldims/"

In [12]:
for tag in list(df.tag):
    tmp = d[d[tag]!=0]
    tmp.reset_index(drop=True,inplace=True)
    tmp = tmp[tmp.columns[(tmp != 0).any()]] # eliminating all zero columns
    
    num_bin = 2 * int(round( np.sqrt( len(tmp) / np.pi ) )) # square that fits the circle
    
    X = tmp.iloc[:,1:].as_matrix()
    tsne_obj = tsne(n_components=2)
    
    subspace_tsne = pd.DataFrame(tsne_obj.fit_transform(X),columns=["x","y"])
    subspace_tsne['grid_x'] = pd.cut(subspace_tsne['x'],num_bin,labels=False)
    subspace_tsne['grid_y'] = pd.cut(subspace_tsne['y'],num_bin,labels=False)
    subspace_tsne['local_path'] = tmp.local_path
    
    grid_side = num_bin 
    px_w = thumb_side * grid_side
    px_h = thumb_side * grid_side
    
    x,y = range(grid_side) * grid_side, np.repeat(range(grid_side),grid_side)
    grid_list = pd.DataFrame(x,columns=['x'])
    grid_list['y'] = y
    
    point = []
    for i in range(len(grid_list)):
        point.append(Point(grid_list.x.loc[i],grid_list.y.loc[i]))

    grid_list['point'] = point
        
    avg_dist = []
    open_grid = list(grid_list.point)
    canvas = Image.new('RGB',(px_w,px_h),(50,50,50))

    plotted = 0
    for i in np.random.permutation(np.arange(np.round(len(subspace_tsne)))):
        im = Image.open(subspace_tsne.local_path.loc[i])
        im.thumbnail((thumb_side,thumb_side),Image.ANTIALIAS)
        x = subspace_tsne.grid_x.loc[i] * thumb_side 
        y = subspace_tsne.grid_y.loc[i] * thumb_side 
        grid_point = Point((subspace_tsne.grid_x.loc[i],subspace_tsne.grid_y.loc[i]))

        if grid_point in open_grid:
            canvas.paste(im,(x,y))
            open_grid.remove(grid_point)
            plotted+=1

        elif len(open_grid) == 0:
            break

        else:
            closest_open = min(open_grid,key=lambda x: grid_point.distance(x))
            if grid_point.distance(closest_open) < crit: # adjacency criterion
                x = int(closest_open.x) * thumb_side
                y = int(closest_open.y) * thumb_side
                canvas.paste(im,(x,y))
                open_grid.remove(closest_open)
                avg_dist.append( float(grid_point.distance(closest_open)) )
                plotted+=1

    proportion = str(int(round(plotted*100/len(subspace_tsne))))
    dims = str(len(tmp.columns)-1)
    canvas.save(PRE+tag+"_"+str(num_bin)+"_"+proportion+"_"+dims+"_"+str(plotted)+".png")

    print str(plotted),proportion,dims,tag,num_bin,np.mean(avg_dist)

1855 72 684 clothing 58 1.0
3897 55 849 person 94 1.0
2495 65 488 hair 70 1.0
1173 80 449 night 44 1.0
906 70 697 plant 40 1.0
2003 66 1220 vehicle 62 1.0
996 73 727 pet 42 1.0
1436 73 586 dish 50 1.0
801 75 314 landmark 36 1.0
1090 80 433 horizon 42 1.0
2005 67 802 meal 62 1.0
1793 66 862 food 58 1.0
814 70 618 label 38 1.0
1101 74 772 flower 44 1.0
889 75 645 furniture 38 1.0
781 73 348 album cover 36 1.0
1214 75 674 art 46 1.0
856 76 698 sculpture 38 1.0
711 70 629 sports 36 1.0
960 75 566 mobile device 40 1.0
976 68 566 painting 42 1.0
818 79 626 modern art 36 1.0
2354 65 530 hairstyle 68 1.0
711 67 723 animal 36 1.0
1464 64 672 document 54 1.0
1088 77 294 blond 42 1.0
1143 71 789 produce 46 1.0
2056 69 1126 toy 62 1.0
786 74 379 nose 36 1.0
1575 67 613 finger 54 1.0
945 72 646 lighting 40 1.0
804 75 738 flooring 36 1.0
882 74 348 plaza 38 1.0
1083 81 477 sea 42 1.0
1294 69 541 profession 48 1.0
1202 81 388 skyline 44 1.0
784 77 596 jewellery 36 1.0
992 77 368 portrait 40 1.0
1982 

In [13]:
path_list = []
for tag in list(df.tag):
    tmp = d[d[tag]!=0]
    path_list.append(list(tmp.local_path))

In [14]:
path_list = [item for sublist in path_list for item in sublist]

In [15]:
path_list = list(set(path_list))

In [16]:
len(path_list)

38694

In [24]:
float(38694)/len(d)

0.8161569289179498

In [22]:
print list(df.tag)

['clothing', 'person', 'hair', 'night', 'plant', 'vehicle', 'pet', 'dish', 'landmark', 'horizon', 'meal', 'food', 'label', 'flower', 'furniture', 'album cover', 'art', 'sculpture', 'sports', 'mobile device', 'painting', 'modern art', 'hairstyle', 'animal', 'document', 'blond', 'produce', 'toy', 'nose', 'finger', 'lighting', 'flooring', 'plaza', 'sea', 'profession', 'skyline', 'jewellery', 'portrait', 'selfie', 'costume', 'black hair', 'hand', 'coast', 'pattern', 'outerwear', 'headgear', 'panorama', 'brown hair']


In [25]:
d.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47410 entries, 0 to 47409
Columns: 3883 entries, local_path to tapir
dtypes: float64(3882), object(1)
memory usage: 1.4+ GB
